# Analysis market transactions (positions)

### Installing all needed libraries

In [8]:
import pathlib
import pandas as pd

### Reading all sources files for analysis

In [9]:
SOURCE_PATH = pathlib.Path("./sources/")
pnl_df = pd.read_excel(SOURCE_PATH / "PnL.xlsx")
positions_detail_df = pd.read_excel(SOURCE_PATH / "position_details.xlsx")
instrument_detail_df = pd.read_json(SOURCE_PATH / "instrument_details.json")